In [21]:
from datetime import datetime
import pandas as pd

In [57]:
def refine_picks(originalName, secondsBetween):
    f = pd.read_csv(originalName+'.txt', sep=' ', header=None)

    #first change all times to strings
    date_str = '%Y-%m-%dT%H:%M:%S.%f'
    
    for row in range(len(f.iloc[:][0])):
        f.loc[row,3] = datetime.strptime(f.iloc[row][3], date_str)

    x = secondsBetween
    
    df = pd.DataFrame(data={'array': [], 'station': [], 'wave type': [], 'datetime':[]})
    Pflag = False
    for row in range(len(f[0][:])):
        # first find a P wave
        if f.iloc[row][2] == 'P': 
            # then make P flag True to check next one
            Pflag = True
        if f.iloc[row][2] == 'S' and Pflag == True:
            # if there is an S wave following a P wave, check time difference
            difference = (f.iloc[row][3] - f.iloc[row-1][3]).total_seconds()
            if difference <= x:
                # add p wave to dataframe
                df.loc[len(df.index)] = [f.iloc[row-1][0], f.iloc[row-1][1], f.iloc[row-1][2], f.iloc[row-1][3]]
                # add s wave to dataframe
                df.loc[len(df.index)] = [f.iloc[row][0], f.iloc[row][1], f.iloc[row][2], f.iloc[row][3]]
        if f.iloc[row][3] == 'S':
            Pflag = False
    df.to_csv(originalName + '_refined.txt')